In [273]:
from tkinter import *
import tkFileDialog
import pandas as pd

ModuleNotFoundError: No module named 'tkFileDialog'

In [177]:
rutasElMundoSalud = list()

In [126]:
def CrearVentana():
    nuevoRoot = Tk()
    nuevoRoot.title("Grupo X - Busqueda de noticias") # Escribir nombre de la ventana
    nuevoRoot.resizable(False, False) # Si se puede escalar a lo ancho y a lo alto respectivamente
    nuevoRoot.iconbitmap("icono256.ico") # Escoger imagen que acompaña al titulo
    return nuevoRoot

In [131]:
def CrearFrame(nuevoColor):
    nuevoFrame = Frame(root) # Asociar a la ventana
    nuevoFrame.config(bg=nuevoColor) #Establecer un color al contenedor   
    # Esta linea siempre tiene que ir al final de la funcion:
    nuevoFrame.pack() # Reescalar el frame cuando se reescale la ventana
    return nuevoFrame

In [280]:
def MostrarNoticia():
    f = open("ElMundo/Salud/salud.2021-12-10.000.txt", "r")
    leer = f.read()
    contenedorNoticia.insert(END, leer)
    f.close()

In [281]:
def BuscarNoticias():
    rutasElMundoSalud = list()
    df = pd.read_csv("rutas.csv")
    for i in range(len(df)):
        if(df["Noticiario"][i]=="ElMundo" and df["Categoria"][i]=="Salud"):
            rutasElMundoSalud.append(df["rutas"][i])
    # Actualizar el desplegable
    opcionesRanking = rutasElMundoSalud
    valorMostradoRanking = StringVar(frameResultados) # Para recoger el dato seleccionado
    valorMostradoRanking.set(opcionesRanking[0]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
    desplegableRanking = OptionMenu(frameResultados, valorMostradoRanking, *opcionesRanking) # Crear el desplegable
    desplegableRanking.grid(row=1, column=1, padx=(0, 20), pady=(20, 10), sticky="sw") # Ubicar el desplegable

In [282]:
root = CrearVentana()  # Crear frame de la ventana
frameConsulta = CrearFrame("#EEEEEE") # Crear contenedor para un grupo de elementos (como un div de HTML)
frameAjustes = CrearFrame("#EEEEEE") # Crear contenedor para un grupo de elementos (como un div de HTML)
frameResultados = CrearFrame("#EEEEEE") # Crear contenedor para un grupo de elementos (como un div de HTML)

labelConsulta = Label(frameConsulta, text="Consulta:").grid(row=0, column=0, padx=10, pady=20, sticky="e") # Crear y ubicar el texto
cuadroConsulta = Entry(frameConsulta).grid(row=0, column=1, padx=10, pady=20, sticky="ew") # Crear y ubicar el cuadro de texto

labelTopN = Label(frameAjustes, text="TOP-N:").grid(row=0, column=0, padx=(20, 0), sticky="e") # Crear y ubicar el texto
opcionesN = [2, 3, 4, 5, 6, 7, 8] # Opciones que se mostrarán en el desplegable de TOP-N
valorMostradoN = StringVar(frameAjustes) # Para recoger el dato seleccionado
valorMostradoN.set(opcionesN[3]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
desplegableN = OptionMenu(frameAjustes, valorMostradoN, *opcionesN) # Crear el desplegable
desplegableN.grid(row=0, column=1, padx=(0, 20), sticky="w") # Ubicar el desplegable

labelFiltrar = Label(frameAjustes, text="Filtrar:").grid(row=0, column=2, padx=(20, 0), sticky="e") # Crear y ubicar el texto
opcionesFiltrar = ["Todos", "El Mundo", "El Pais", "20 minutos"] # Opciones que se mostrarán en el desplegable de Filtrar
valorMostradoFiltrar = StringVar(frameAjustes) # Para recoger el dato seleccionado
valorMostradoFiltrar.set(opcionesFiltrar[0]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
desplegableFiltrar = OptionMenu(frameAjustes, valorMostradoFiltrar, *opcionesFiltrar) # Crear el desplegable
desplegableFiltrar.grid(row=0, column=3, padx=(0, 20), sticky="w") # Ubicar el desplegable

botonBuscar = Button(frameAjustes, text="Buscar", command=BuscarNoticias).grid(row=0, column=4, padx=20, pady=10) # Crear y ubicar el botón

labelRanking = Label(frameResultados, text="Ranking:").grid(row=1, column=0, padx=(20, 0), pady=(20, 10), sticky="se") # Crear y ubicar el texto
opcionesRanking = ["Noticia 1 - SIM 88%", "Noticia 2 - SIM 85%", "Noticia 3 - SIM 78%", "Noticia 4 - SIM 68%", "Noticia 5 - SIM 60%"] # Opciones que se mostrarán en el desplegable del ranking
valorMostradoRanking = StringVar(frameResultados) # Para recoger el dato seleccionado
valorMostradoRanking.set(opcionesRanking[0]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
desplegableRanking = OptionMenu(frameResultados, valorMostradoRanking, *opcionesRanking) # Crear el desplegable
desplegableRanking.grid(row=1, column=1, padx=(0, 20), pady=(20, 10), sticky="sw") # Ubicar el desplegable

botonBuscar = Button(frameResultados, text="Leer noticia", command=MostrarNoticia).grid(row=2, column=0, columnspan=2, padx=20, pady=10, sticky="new") # Crear y ubicar el botón

labelNoticia = Label(frameResultados, text="Texto de la noticia:").grid(row=0, column=2, sticky="w", padx=20, pady=(20, 0)) # Crear y ubicar el texto
contenedorNoticia = Text(frameResultados, width=50, height=14, padx=10) # Crear el cuadro de texto de varias filas
scrollNoticia = Scrollbar(frameResultados, command=contenedorNoticia.yview) # Crear y escalar el scroll, y asociarlo a su cuadro de texto
scrollNoticia.grid(row=1, column=3, rowspan=2, padx=(0, 20), pady=(0, 20), sticky="nsw") # Ubicar el scroll
contenedorNoticia.grid(row=1, column=2, rowspan=2, padx=(20, 0), pady=(0, 20)) # Ubicar el cuadro de texto de varias filas
contenedorNoticia.config(yscrollcommand=scrollNoticia.set) # Para que el scroll muestre a qué altura se encuentra el cuadro de texto asociado

root.mainloop() # Bucle infinito para escuchar acciones del usuario (siempre tiene que ir al final)

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Pablo\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Pablo\AppData\Local\Temp/ipykernel_15484/1163960695.py", line 5, in MostrarNoticia
    print(File2.read())
NameError: name 'File2' is not defined
